In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import scipy as sp
import scipy.stats
from tqdm import tqdm
import itertools
import re
res_digit = r'[0-9]'


from sklearn.metrics import classification_report

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

from sklearn import preprocessing 


from sklearn.model_selection import RandomizedSearchCV 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GroupKFold
from skopt import BayesSearchCV
# parameter ranges are specified by one of below
from skopt.space import Real, Categorical, Integer


In [2]:
# This is a hack to make the library in the parent folder available for imoprts
# A better solution is by np8 here:
# https://stackoverflow.com/questions/714063/importing-modules-from-parent-folder
import sys
import os
import inspect

thisdir = sys.path[0]
print(f"thisdir = {thisdir}")
parentdir = os.path.dirname(thisdir)
#print(f"parentdir = {parentdir}")
if not parentdir in sys.path:
    print("Adding parent directory to python path")
    sys.path.insert(1, parentdir)
else:
    print("Skipping adding parent direct to path (there already)")

print(f"sys.path =\n{sys.path}")



thisdir = /home/luke/git/external/predicament/notebooks
Adding parent directory to python path
sys.path =
['/home/luke/git/external/predicament/notebooks', '/home/luke/git/external/predicament', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/luke/.local/lib/python3.10/site-packages', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/lib/python3.10/dist-packages']


In [3]:
## ensure relative path to data directory is sound
# for the notebook we need to modify the BASE_DATA_FOLDER
import os 
os.environ['PREDICAMENT_DATA_DIR'] =  '../data'

from predicament.utils.config import FEATURED_BASE_PATH

In [4]:
from predicament.utils.file_utils import load_dataframe_and_config

from predicament.data.features import IDEAL_FEATURE_GROUP

from predicament.evaluation.balancing import get_group_label_counts
from predicament.evaluation.balancing import balance_data
from predicament.evaluation.grouping import get_group_assignments
from predicament.evaluation.staging import get_design_matrix
from predicament.evaluation.results import output_model_best_from_results
from predicament.evaluation.results import save_results_df_to_file

from predicament.evaluation.hyperparameters import get_param_scopes
from predicament.evaluation.hyperparameters import get_param_search_object

from predicament.models.mlp_wrappers import ThreeHiddenLayerClassifier


In [5]:
# high level choices
subdir = 'dreem_10secs' # dataset
held_out = 'participant'
is_balanced = True # balance data set
use_only_ideal_features = True # restrict to preferred ideal features
standardise_data = True
n_iter = 50 # number of iterations for your search
new_search = True # restarts the search
random_state = 43


## Load featured data and balance if required

Before running this, you will need to generate featured data. See README file for details. For the variable `subdir` above replace this with the subdirectory name of the featured data.

In [6]:
featured_data_dir = os.path.join(FEATURED_BASE_PATH,subdir)

featured_df, featured_config = load_dataframe_and_config(
    featured_data_dir, 'featured.csv')
n_channels = int(featured_config['LOAD']['n_channels'])
data_format = featured_config['LOAD']['data_format']
channels = json.loads(featured_config['LOAD']['channels'].replace("'",'"'))
participant_list = json.loads(featured_config['LOAD']['participant_list'].replace("'",'"'))
Fs = int(featured_config['LOAD']['sample_rate'])
window_size = int(featured_config['LOAD']['window_size'])
time = window_size/Fs
print(f"Fs: {Fs}, n_samples = {window_size}, time: {time}s, n_channels: {n_channels}")


Fs: 250, n_samples = 2560, time: 10.24s, n_channels: 4


In [7]:
if is_balanced:
    # balance featured data
    subject_condition_counts = get_group_label_counts(featured_df, 'participant', 'condition')
    print(f"before balancing: subject_condition_counts = {subject_condition_counts}")
    featured_df = balance_data(featured_df, group_col='participant', label_col='condition')
    subject_condition_counts = get_group_label_counts(featured_df, 'participant', 'condition')

    print(f"after balancing: subject_condition_counts = {subject_condition_counts}")

before balancing: subject_condition_counts = [[321. 180. 180. 180. 180.]
 [321. 133. 180. 219. 180.]
 [321.  39. 133. 180. 133.]
 [321.  86. 180. 180. 180.]
 [321. 227. 133.  86. 180.]
 [  0.  39.   0. 414. 133.]
 [328. 164.   0. 197. 141.]
 [321. 227. 180. 133.  30.]
 [321. 227.   0. 133. 133.]
 [317. 125.   0. 216. 135.]
 [368. 180. 133. 180. 133.]
 [274. 227. 180. 133. 180.]]
after balancing: subject_condition_counts = [[119. 127. 180. 111. 116.]
 [119. 127. 180. 111. 116.]
 [119.  39. 133. 111. 116.]
 [119.  86. 180. 111. 116.]
 [119. 127. 133.  86. 116.]
 [  0.  39.   0. 111. 116.]
 [119. 127.   0. 111. 116.]
 [119. 127. 180. 111.  30.]
 [119. 127.   0. 111. 116.]
 [119. 125.   0. 111. 116.]
 [119. 127. 133. 111. 116.]
 [119. 127. 180. 111. 116.]]


## Define model and hyperparamer search

In [8]:
# the base model to tune
# estimator = SVC()
#estimator = RandomForestClassifier()
# max_iter_opt = 1
# estimator = MLPClassifier(max_iter=max_iter_opt)
estimator = ThreeHiddenLayerClassifier()
print(f"estimator = {estimator}")
# search_type = 'random_search'
search_type = 'bayesian_optimization'

excludes = ['layer3']
#excludes = ['layer2', 'layer3']
overrides = dict()

# now create the parameter search object and run the hyperparameter search
param_scopes = get_param_scopes(
    search_type, estimator, excludes=excludes, **overrides)
print(f"param_scopes = {param_scopes}")


estimator = None
param_scopes = {'layer1': Integer(low=10, high=100, prior='uniform', transform='identity'), 'layer2': Integer(low=10, high=100, prior='uniform', transform='identity'), 'activation': Categorical(categories=('tanh', 'relu'), prior=None), 'solver': Categorical(categories=('sgd', 'adam'), prior=None), 'alpha': Real(low=1e-06, high=0.01, prior='log-uniform', transform='identity'), 'learning_rate': Categorical(categories=('constant', 'adaptive'), prior=None), 'learning_rate_init': Real(low=1e-06, high=10.0, prior='log-uniform', transform='identity')}


## Define data properties and data-split 

In [9]:
feature_set = json.loads(featured_config['FEATURED']['feature_set'].replace("'",'"'))
if use_only_ideal_features:
    feature_set = list(IDEAL_FEATURE_GROUP.intersection(feature_set))
    
print(f"feature_set = {feature_set}")

# extract input data
# use all features in file
feature_types, feature_names, designmtx = get_design_matrix(
    featured_df, feature_set)
# extract labels
labels = featured_df['condition'].values.astype(int)

if standardise_data:
    scaler = preprocessing.StandardScaler().fit(designmtx)
    designmtx = scaler.transform(designmtx)
    
# prepare Hold one group out cross validation
held_out, groups, group_assignments = get_group_assignments(featured_df)
n_groups = len(groups)
# cross validation splits    
group_kfold = GroupKFold(n_splits=n_groups)

feature_set = ['MAD', 'IQR', 'MeanFreq', 'arCoeff', 'Hurst', 'Mean', 'SD', 'MaxFreqInd', 'Max', 'Min', 'FreqKurtosis', 'Correlation', 'LyapunovExponent', 'LempelZivComplexity', 'FreqSkewness']


## Define and Execute hyperparameter search strategy 

In [10]:

def create_callback_and_storage(param_search):
    intermediate_results = []
    # Define a custom callback function to store intermediate results
    def on_step(optim_result):
        # Store the current state of the optimization process
        intermediate_results.append((optim_result.func_vals, optim_result.x_iters))

        # Print out the best score and best parameters found so far
        best_score = -optim_result.fun
        print("Best score: %s" % best_score)
        print("Best parameters: %s" % optim_result.x)
    return on_step, intermediate_results


param_search  = get_param_search_object(
    search_type, estimator, param_scopes=param_scopes, 
    n_iter = n_iter, cv=group_kfold,
    verbose=2, random_state=random_state, n_jobs=-1,
    fit_params={'X': designmtx, 'y': labels, 'callbacks': None},
    refit=False  # Ensure that the search does not refit the model with the best parameters found so far
)

if new_search:
    on_step, intermediate_results = create_callback_and_storage(param_search)
else:
    param_search.fit_params['search_results'] = intermediate_results
try:
    # Fit the random search model
    _ = param_search.fit(X=designmtx, y=labels, groups=group_assignments, callback=on_step)
except Exception as e:
    print("Exception occurred:", str(e))
finally:
    # Print or process intermediate results even if an error occurs
    print("Intermediate Results:")
    for i, (scores, params) in enumerate(intermediate_results):
        print(f"Iteration {i + 1}: Scores - {scores}, Params - {params}")
    new_search = False
    random_state = np.random.randint(100000)


Fitting 12 folds for each of 1 candidates, totalling 12 fits
Best score: 0.21999412263988868
Best parameters: ['tanh', 1.0310736334774332e-05, 30, 49, 'constant', 0.3660896898602607, 'sgd']
Fitting 12 folds for each of 1 candidates, totalling 12 fits


/home/luke/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/luke/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/luke/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/luke/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/luke/.local/lib/python3.10/site-packages/sklearn/n

Best score: 0.21999412263988868
Best parameters: ['tanh', 1.0310736334774332e-05, 30, 49, 'constant', 0.3660896898602607, 'sgd']
Fitting 12 folds for each of 1 candidates, totalling 12 fits


/home/luke/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/luke/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/luke/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/luke/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/luke/.local/lib/python3.10/site-packages/sklearn/n

Best score: 0.2363425370218074
Best parameters: ['relu', 0.00012282687710642398, 22, 83, 'constant', 0.007096604364095074, 'sgd']
Fitting 12 folds for each of 1 candidates, totalling 12 fits


...

/home/luke/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Best score: 0.26505663463720097
Best parameters: ['relu', 1.255403047870767e-06, 15, 90, 'constant', 0.0001392593087733178, 'adam']
Fitting 12 folds for each of 1 candidates, totalling 12 fits


/home/luke/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/home/luke/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/home/luke/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/home/luke/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


Intermediate Results:
Iteration 1: Scores - [-0.21999412], Params - [['tanh', 1.0310736334774332e-05, 30, 49, 'constant', 0.3660896898602607, 'sgd'], ['relu', 0.005885338806523476, 51, 61, 'constant', 1.4862038462901743e-06, 'adam'], ['relu', 0.00012282687710642398, 22, 83, 'constant', 0.007096604364095074, 'sgd'], ['relu', 8.936224314764104e-06, 53, 79, 'constant', 0.00015458417467371826, 'sgd'], ['relu', 0.00014688992790007224, 55, 43, 'constant', 0.0014134459655991956, 'sgd'], ['relu', 4.468330777342982e-05, 48, 30, 'constant', 1.0584035163532994, 'sgd'], ['relu', 2.442583506253724e-06, 73, 85, 'adaptive', 4.754624851184224e-06, 'sgd'], ['tanh', 0.00034973710968534325, 11, 80, 'constant', 2.1512335328024205e-06, 'adam'], ['tanh', 3.487488243540924e-06, 15, 16, 'adaptive', 0.00010486981381071045, 'sgd'], ['tanh', 0.0007356479814621087, 74, 85, 'constant', 0.0015143842081374423, 'adam'], ['relu', 0.006816267225065473, 15, 68, 'constant', 3.128194906850533, 'sgd'], ['relu', 1.764340676

KeyboardInterrupt: 

In [ ]:
param_search.get_params()

In [ ]:
param_search.result

## Saving and outputing results

In [ ]:
result_df = pd.DataFrame(param_search.cv_results_)
i = 0
result_df.insert(i, 'model', str(estimator))
i +=1
result_df.insert(i, 'data format', data_format)
i +=1
result_df.insert(i, 'held out', held_out)
i +=1
result_df.insert(i, 'balanced', is_balanced)
i +=1
result_df.insert(i, 'n_splits', param_search.get_params()['cv'].get_n_splits())
i +=1
result_df.insert(i, 'feature set', str(feature_types))
i +=1
result_df.insert(i, 'window size', window_size)
display(result_df)
results_fname = f'{search_type}_{str(estimator)}'
print(f"Saving to {results_fname}")
save_results_df_to_file(result_df, results_fname)

In [ ]:
_ = output_model_best_from_results(result_df)

In [ ]:
thresholds = np.linspace(0,0.25,51)
N = len(result_df)
props = np.empty(thresholds.size)
for t, thresh in enumerate(thresholds):
    count = np.sum(result_df['mean_test_score'] > thresh)
    props[t] = count/N
plt.plot(thresholds, props)
plt.xlabel("mean test score")
plt.ylabel("proportion greater than")

In [ ]:
result_df.columns

In [ ]:
display_cols = [col for col in result_df.columns if col.startswith('param') or (col =='mean_test_score')]
result_df[result_df['mean_test_score'] >= 0.225][display_cols]

In [ ]:
# or just show all rows
result_df[display_cols]